In [1]:
import pandas as pd

# Load the dataset
# Make sure 'spam.csv' is in the same directory as your Jupyter Notebook
try:
    df = pd.read_csv('spam.csv', encoding='latin-1')
except UnicodeDecodeError:
    # Some versions might need a different encoding
    df = pd.read_csv('spam.csv', encoding='ISO-8859-1')
except FileNotFoundError:
    print("Error: 'spam.csv' not found. Make sure it's in the correct directory.")
    # If the file has a different name, update it here, e.g., 'SMSSpamCollection'
    # df = pd.read_csv('SMSSpamCollection', sep='\t', names=['label', 'message'])


# Display the first few rows of the dataset
print("Dataset loaded successfully. First 5 rows:")
print(df.head())

# Display basic info about the dataset
print("\nDataset Info:")
df.info()

# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())

Dataset loaded successfully. First 5 rows:
     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 

In [2]:
# Drop the unnecessary columns
# Assuming the extra columns are Unnamed: 2, Unnamed: 3, Unnamed: 4
df = df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], errors='ignore')

# Rename the columns for clarity
df = df.rename(columns={'v1': 'label', 'v2': 'message'})

# Display the first few rows of the cleaned dataset
print("Dataset after cleaning and renaming columns:")
print(df.head())

# Check the new column names and info
print("\nCleaned Dataset Info:")
df.info()

Dataset after cleaning and renaming columns:
  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

Cleaned Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [3]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
encoder = LabelEncoder()

# Fit and transform the 'label' column
df['label'] = encoder.fit_transform(df['label'])

# Display the first few rows to see the encoded labels
print("Dataset after encoding 'label' column:")
print(df.head())

# Display the unique encoded labels
print("\nUnique encoded labels:")
print(df['label'].unique())

# Check the count of 'ham' (0) and 'spam' (1)
print("\nLabel counts:")
print(df['label'].value_counts())

Dataset after encoding 'label' column:
   label                                            message
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...

Unique encoded labels:
[0 1]

Label counts:
label
0    4825
1     747
Name: count, dtype: int64


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Initialize TF-IDF Vectorizer
# max_features limits the number of features (words) to consider, common for text data
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the 'message' column
# X will be the features (vectorized messages)
X = tfidf_vectorizer.fit_transform(df['message']).toarray()

# y will be the labels (encoded spam/ham)
y = df['label']

# Split the data into training and testing sets
# test_size=0.2 means 20% of data for testing, 80% for training
# random_state ensures reproducibility of the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Text Vectorization and Data Splitting complete.")
print(f"Shape of X_train (training features): {X_train.shape}")
print(f"Shape of X_test (testing features): {X_test.shape}")
print(f"Shape of y_train (training labels): {y_train.shape}")
print(f"Shape of y_test (testing labels): {y_test.shape}")

Text Vectorization and Data Splitting complete.
Shape of X_train (training features): (4457, 5000)
Shape of X_test (testing features): (1115, 5000)
Shape of y_train (training labels): (4457,)
Shape of y_test (testing labels): (1115,)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize the Multinomial Naive Bayes model
model = MultinomialNB()

# Train the model using the training data
model.fit(X_train, y_train)

print("Model training complete.")

# --- Model Evaluation (Next Part) ---
# Predict on the test data
y_pred = model.predict(X_test)

print("\nModel Evaluation:")

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Calculate precision
precision = precision_score(y_test, y_pred)
print(f"Precision (Spam): {precision:.4f}") # Precision for class 1 (spam)

# Calculate recall
recall = recall_score(y_test, y_pred)
print(f"Recall (Spam): {recall:.4f}") # Recall for class 1 (spam)

# Calculate F1-score
f1 = f1_score(y_test, y_pred)
print(f"F1-Score (Spam): {f1:.4f}") # F1-Score for class 1 (spam)

# Generate Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Plot Confusion Matrix for better visualization
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Ham (Predicted)', 'Spam (Predicted)'],
            yticklabels=['Ham (Actual)', 'Spam (Actual)'])
plt.xlabel('Predicted Label')
plt.ylabel('Actual Label')
plt.title('Confusion Matrix')
plt.show()